In [2]:
import numpy as np
initial = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

In [3]:
def win(a, b):
    return b/(a+b)

In [6]:
def calc(arrangement):
    dp = np.zeros(shape=(5, 16))
    dp[0] = 1
    for layer in range(1, 5):
        group_size = 1<<(layer-1) # the size of the group above layer
        for i in range(16):
            current_individual = arrangement[i]
            current_group = i // group_size
            if current_group % 2 == 0:
                competition_group = current_group + 1
            else:
                competition_group = current_group - 1
            competitor_index_range = range(competition_group * group_size, (competition_group+1)*group_size)
            for competitor_index in competitor_index_range:
                competitor = arrangement[competitor_index]
                dp[layer][i] += win(current_individual, competitor) * dp[layer-1][competitor_index]
            dp[layer][i] *= dp[layer-1][i]
    print(np.array2string(dp.round(2), max_line_width=np.inf))
    return dp[4][arrangement.index(2)]
calc(initial)

[[1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.  ]
 [0.94 0.06 0.53 0.47 0.71 0.29 0.76 0.24 0.65 0.35 0.82 0.18 0.59 0.41 0.88 0.12]
 [0.84 0.02 0.08 0.06 0.36 0.09 0.47 0.08 0.26 0.1  0.58 0.06 0.16 0.09 0.71 0.04]
 [0.7  0.01 0.03 0.02 0.09 0.01 0.13 0.01 0.09 0.02 0.29 0.01 0.06 0.03 0.48 0.01]
 [0.52 0.   0.01 0.01 0.03 0.   0.06 0.   0.02 0.   0.11 0.   0.01 0.   0.22 0.  ]]


0.2160396878170165

In [75]:
print(f"{calc(initial)}\t{initial}")
best = (calc(initial), initial)
for i in range(16):
    for j in range(i+1, 16):
        new_arrangement = initial.copy()
        new_arrangement[i], new_arrangement[j] = new_arrangement[j], new_arrangement[i]
        prob = calc(new_arrangement)
        if prob > best[0]:
            print(f"{prob}\t{new_arrangement}")
            best = (prob, new_arrangement)
            
print("Best:")
print(f"{best[0]}\t{best[1]}")   

0.2160396878170165	[1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
0.2204033628021303	[8, 16, 1, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
0.2209216666385926	[9, 16, 8, 1, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
0.22374757237230697	[5, 16, 8, 9, 1, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
0.2343402654712729	[12, 16, 8, 9, 5, 1, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
0.2366648898645163	[13, 16, 8, 9, 5, 12, 4, 1, 6, 11, 3, 14, 7, 10, 2, 15]
0.2396915314494737	[14, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 1, 7, 10, 2, 15]
0.2816191915195929	[1, 3, 8, 9, 5, 12, 4, 13, 6, 11, 16, 14, 7, 10, 2, 15]
Best:
0.2816191915195929	[1, 3, 8, 9, 5, 12, 4, 13, 6, 11, 16, 14, 7, 10, 2, 15]


# An experiment
What if you repeatedly search and apply the optimal swap?

In [69]:
# let's do some experiments
# find optimal overall shuffle
import random

def find_next_best_arrangement(prev):
    best = (0, prev)
    for i in range(16):
        for j in range(i, 16):
            new_arrangement = prev.copy()
            new_arrangement[i], new_arrangement[j] = new_arrangement[j], new_arrangement[i]
            prob = calc(new_arrangement)
            if prob > best[0]:
                best = (prob, new_arrangement)
    return best[1]
            

for i in range(15):
    random.shuffle(initial)
    next_arrangement = find_next_best_arrangement(initial)
    while next_arrangement != initial:
        initial = next_arrangement
        next_arrangement = find_next_best_arrangement(initial)
    print(f"{calc(initial)}\t{initial}")

0.33652684250579035	[12, 13, 10, 11, 2, 16, 14, 15, 7, 8, 6, 9, 4, 5, 1, 3]
0.33652684250579035	[12, 13, 10, 11, 16, 2, 14, 15, 8, 7, 9, 6, 1, 3, 5, 4]
0.33652684250579035	[12, 13, 10, 11, 16, 2, 14, 15, 5, 4, 3, 1, 9, 6, 8, 7]
0.33652684250579035	[5, 4, 1, 3, 7, 8, 6, 9, 14, 15, 2, 16, 13, 12, 10, 11]
0.33652684250579035	[2, 16, 15, 14, 12, 13, 10, 11, 6, 9, 8, 7, 3, 1, 4, 5]
0.33652684250579035	[1, 3, 5, 4, 8, 7, 9, 6, 12, 13, 10, 11, 14, 15, 2, 16]
0.33652684250579035	[12, 13, 10, 11, 16, 2, 15, 14, 5, 4, 1, 3, 8, 7, 9, 6]
0.33652684250579035	[13, 12, 10, 11, 14, 15, 16, 2, 5, 4, 1, 3, 9, 6, 8, 7]
0.33652684250579024	[7, 8, 9, 6, 5, 4, 1, 3, 11, 10, 12, 13, 15, 14, 2, 16]
0.33652684250579035	[7, 8, 6, 9, 4, 5, 3, 1, 15, 14, 16, 2, 13, 12, 10, 11]
0.33652684250579035	[12, 13, 10, 11, 2, 16, 15, 14, 5, 4, 1, 3, 8, 7, 9, 6]
0.33652684250579035	[4, 5, 1, 3, 8, 7, 9, 6, 14, 15, 16, 2, 12, 13, 10, 11]
0.33652684250579024	[7, 8, 6, 9, 4, 5, 3, 1, 10, 11, 12, 13, 16, 2, 15, 14]
0.3365268425

It seems like you end up at an optimal solution after repeated swaps >99% of the time, but not always. Quite a surprising result.